In [2]:
import mne 
import os 
from utils import *

In [4]:
def fif_read(root,file_name):
    raw_path = os.path.join(root,file_name)
    raw = mne.io.read_raw(raw_path)
    raw.load_data()
    raw.plot()
    # raw.filter(l_freq=4,h_freq=40)

    # events, event_id = mne.events_from_annotations(raw)
    # print(events)
    # tmin= 0 
    # tmax = 1
    # baseline = (None, 0)
    # epochs_ica = mne.Epochs(raw,
    #                         events = events,)

In [5]:
files = os.listdir(fif_dir)
i=0
# for f in files:
#     fif_read(fif_dir, f)
#     print(i)
#     i=i+1

fif_read(fif_dir,files[i])

Opening raw data file /media/pankhi/57A77BEF00BB01D2/DATA/BCI/fif_data/EEG_session1_sub10_multigrasp_realMove.fif...


/tmp/ipykernel_36025/2660248441.py:3: RuntimeWarning: This filename (/media/pankhi/57A77BEF00BB01D2/DATA/BCI/fif_data/EEG_session1_sub10_multigrasp_realMove.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw(raw_path)


Isotrak not found
    Range : 0 ... 5250599 =      0.000 ...  2100.240 secs
Ready.
Reading 0 ... 5250599  =      0.000 ...  2100.240 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 4125 samples (1.650 s)

[]
